#Getting and knowing pandas and pyspark with chiptole dataset

In [0]:
import pandas as pd

spark


SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
pd

Out[3]: <module 'pandas' from '/databricks/python/lib/python3.9/site-packages/pandas/__init__.py'>

In [0]:
df = pd.read_csv("https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv", sep='\t')

In [0]:
s_df = spark.createDataFrame(df)
sdf = s_df

In [0]:
s_df.show(10)

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                null|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                null|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |
|       3|       1|       Side of Chips|                null|    $1.69 |
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|   $11.75 |
|       4|       1|    Steak Soft Tacos|[Tomatillo Green ...|    $9.25 |
|       5|       1|       Steak Burrito|[Fresh Tomato Sal...|    $9.25 |
+--------+--------+--------------------+-----------

In [0]:
df.head(10)

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",$10.98
6,3,1,Side of Chips,NaN,$1.69
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",$11.75
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",$9.25
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",$9.25


In [0]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   order_id            4622 non-null   int64 
 1   quantity            4622 non-null   int64 
 2   item_name           4622 non-null   object
 3   choice_description  3376 non-null   object
 4   item_price          4622 non-null   object
dtypes: int64(2), object(3)
memory usage: 180.7+ KB


In [0]:
sdf.summary().show()

+-------+-----------------+------------------+-----------------+--------------------+----------+
|summary|         order_id|          quantity|        item_name|  choice_description|item_price|
+-------+-----------------+------------------+-----------------+--------------------+----------+
|  count|             4622|              4622|             4622|                3376|      4622|
|   mean|927.2548680225011|1.0757247944612722|             null|                null|      null|
| stddev|528.8907955866094|0.4101863342575332|             null|                null|      null|
|    min|                1|                 1|6 Pack Soft Drink|[Adobo-Marinated ...|    $1.09 |
|    25%|              477|                 1|             null|                null|      null|
|    50%|              926|                 1|             null|                null|      null|
|    75%|             1393|                 1|             null|                null|      null|
|    max|             1834|   

In [0]:

print(sdf.count())
sdf.columns

4622
Out[10]: ['order_id', 'quantity', 'item_name', 'choice_description', 'item_price']

#Which was the most-ordered item?


In [0]:
df.groupby(['item_name']).agg({'quantity':'sum'}).sort_values(by='quantity',ascending=False).head(1)

,quantity
item_name,
Chicken Bowl,761


In [0]:
from pyspark.sql.functions import sum, avg, count, desc

sdf.groupBy('item_name').agg(sum('quantity').alias('total')).orderBy(desc('total')).show(1) 
#sdf.orderBy("category", desc("price"))


+------------+-----+
|   item_name|total|
+------------+-----+
|Chicken Bowl|  761|
+------------+-----+
only showing top 1 row



#What was the most ordered item in the choice_description column?


In [0]:
df.groupby('choice_description').agg({'choice_description':'count'}).rename(columns={'choice_description':'total1'}).sort_values('total1', ascending=False).head(1)

,total1
choice_description,
[Diet Coke],134


In [0]:
sdf.groupBy('choice_description').agg(count('choice_description').alias("total")).orderBy(desc('total')).show(1)

+------------------+-----+
|choice_description|total|
+------------------+-----+
|       [Diet Coke]|  134|
+------------------+-----+
only showing top 1 row



#How many items were orderd in total?

In [0]:
df['quantity'].sum()

Out[15]: 4972

In [0]:
sdf.select(sum(sdf['quantity'])).show()

+-------------+
|sum(quantity)|
+-------------+
|         4972|
+-------------+



Turn the item price into a float


In [0]:
# df['item_price'][1][1:-1]
remove_dollar = lambda x : float(x[1:-1])
df['item_priced'] = df.item_price.apply(remove_dollar)
df

,order_id,quantity,item_name,choice_description,item_price,item_priced
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39,2.39
1,1,1,Izze,[Clementine],$3.39,3.39
2,1,1,Nantucket Nectar,[Apple],$3.39,3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39,2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98,16.98
...,...,...,...,...,...,...
4617,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ...",$11.75,11.75
4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese...",$11.75,11.75
4619,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",$11.25,11.25
4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",$8.75,8.75


In [0]:
df['item_priced'].dtype

Out[18]: dtype('float64')

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

sdf.withColumn('item_priced', substring('item_price', 2, 5)).show()
# d.show()
# second way
sdf = sdf.withColumn('item_priced', regexp_replace('item_price', '\$', ""))
sdf.withColumn('item_priced', sdf['item_priced'].cast(DoubleType())).show()


+--------+--------+--------------------+--------------------+----------+-----------+
|order_id|quantity|           item_name|  choice_description|item_price|item_priced|
+--------+--------+--------------------+--------------------+----------+-----------+
|       1|       1|Chips and Fresh T...|                null|    $2.39 |      2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |      3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |      3.39 |
|       1|       1|Chips and Tomatil...|                null|    $2.39 |      2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |      16.98|
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |      10.98|
|       3|       1|       Side of Chips|                null|    $1.69 |      1.69 |
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|   $11.75 |      11.75|
|       4|       1|    Steak Soft Tacos|[Tomatillo Green ...|    

#How much was the revenue for the period in the dataset

In [0]:
df.item_priced.sum()

Out[100]: 34500.16

In [0]:
sdf.select(sum(sdf['item_priced'])).show()


+------------------+
|  sum(item_priced)|
+------------------+
|34500.159999999894|
+------------------+



#How many orders were made in the period?

In [0]:
df.order_id.unique().size
# 
df.order_id.nunique()
# or
df_total_orders = df.order_id.value_counts().count()
df_total_orders

Out[126]: 1834

In [0]:
sdf_total_orders = sdf.select('order_id').distinct().count()
sdf_total_orders

Out[125]: 1834

# What is the average revenue amount per order?

In [0]:
df['revenue']=df['quantity']*df['item_priced']
# df['revenue'].sum()/df_total_orders
df.groupby(by=['order_id']).sum().mean()


Out[137]: quantity        2.711014
item_priced    18.811429
revenue        21.394231
dtype: float64

In [0]:
sdf.withColumn('revenue', sdf['quantity']*sdf['item_priced']).agg({'revenue':'sum'}).collect()[0][0]/sdf_total_orders

Out[147]: 21.394231188658598

#How many different items are sold?

In [0]:
df.item_name.value_counts().count()

Out[159]: 50

In [0]:
sdf.select('item_name').distinct().count()

Out[161]: 50